In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import io
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

load_dotenv()

QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL')

qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

s:\Projects\vehicle-insurance-backend-api\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# collection_name = "car_data"  
collection_name = "car_data_modelbrand"  

# Load the sentence transformer model
model = SentenceTransformer("intfloat/multilingual-e5-small")

# Get the embedding dimension
sample_text = "Sample text for dimension check"
sample_embedding = model.encode(sample_text)
embedding_dim = len(sample_embedding)
print(f"Embedding dimension: {embedding_dim}")

Embedding dimension: 384


# Vector Search

In [2]:
from rapidfuzz import fuzz

collection_name = "car_data_modelbrand"  

# Function to search car brands with typo tolerance
def search_car_brand(query, top_k=10):
    # Add context to the query to match our embeddings
    query_with_context = f"car brand: {query}" 
    query_vector = model.encode(query_with_context)
    
    # Use query_filter instead of filter parameter
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k * 3,
        query_filter={
            "must": [
                {
                    "key": "vector_type",
                    "match": {
                        "value": "brand"
                    }
                }
            ]
        }
    )

    # Remove duplicates based on car_brand
    unique_results_brand = {}
    for result in search_result:
        car_model = result.payload['car_brand']
        if car_model not in unique_results_brand:
            unique_results_brand[car_model] = result
    
    # Return top_k unique results
    return list(unique_results_brand.values())[:top_k]


# Function to search car models with typo tolerance
def search_car_model(query, top_k=10):
    # Add generic context to the query to match our embeddings
    query_with_context = f"car model: {query}"  
    query_vector = model.encode(query_with_context)
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k * 3,
        query_filter={
            "must": [
                {
                    "key": "vector_type",
                    "match": {
                        "value": "model"
                    }
                }
            ]
        }
    )
    
    # Remove duplicates based on car_model
    unique_results_model = {}
    for result in search_result:
        car_model = result.payload['car_model']
        if car_model not in unique_results_model:
            unique_results_model[car_model] = result
    
    # Return top_k unique results
    return list(unique_results_model.values())[:top_k]

# Hybrid Search + Normalization (for fuzzy)

In [3]:
from rapidfuzz import process as rapidfuzz_process
from rapidfuzz import fuzz
import re

def normalize_text(s):
    """Normalize text to improve matching"""
    if not isinstance(s, str):
        return s
    s = s.strip().lower()
    s = re.sub(r'[\-–—_/]', ' ', s)  
    s = re.sub(r'\s+', ' ', s)     
    return s  

def hybrid_search(query, choices, vector_type="brand", fuzzy_threshold=75, top_k=3, search_model = None):
    """
    Hybrid search that combines RapidFuzz and embeddings:
    1. Try fuzzy matching first with 75% threshold (fast + handles typos)
    2. If no good fuzzy matches, fall back to embeddings (semantic understanding)
    
    Args:
        query: The search query
        choices: List of choices to search against
        vector_type: "brand" or "model"
        fuzzy_threshold: Minimum score (0-100) for fuzzy matches
        top_k: Number of results to return
    """
    # Normalize query for better matching
    query_norm = normalize_text(query)

    model_to_use = search_model if search_model is not None else model

    
    # Adjust scorer based on query characteristics
    if ' ' in query_norm or len(query_norm) > 10:
        scorer = fuzz.token_sort_ratio  # Better for word order/spacing differences
    else:
        scorer = fuzz.ratio  # Standard for character-level typos
    
    # Step 1: Try RapidFuzz first (faster than embeddings)
    fuzzy_matches = rapidfuzz_process.extract(
        query_norm, 
        [normalize_text(c) for c in choices],  # Normalize choices too
        scorer=scorer,
        limit=top_k * 2  # Get more candidates for filtering
    )
    
    # Map normalized choices back to original labels
    norm_to_orig = {normalize_text(c): c for c in choices}
    fuzzy_matches = [(norm_to_orig.get(match, match), score, idx) for match, score, idx in fuzzy_matches]
    
    # Filter matches that meet our threshold
    good_fuzzy_matches = [(match, score) for match, score, _ in fuzzy_matches if score >= fuzzy_threshold]
    
    results = []
    
    # If we have good fuzzy matches, return those
    if good_fuzzy_matches:
        print(f"Found {len(good_fuzzy_matches)} good fuzzy matches for '{query}'")
        for match, score in good_fuzzy_matches:
            results.append({
                "text": match,
                "score": score / 100.0,  # Normalize to 0-1 scale
                "source": "fuzzy"
            })
    
    # Step 2: If not enough good fuzzy matches, use embeddings
    if len(results) == 0:
        print(f"No good fuzzy matches above threshold {fuzzy_threshold}, using embeddings")
        
        # Use appropriate search function based on vector_type
        if vector_type == "brand":
            # Modify how you call search_car_brand to use the provided model
            query_with_context = f"car brand: {query}"
            query_vector = model_to_use.encode(query_with_context)
            
            # Call Qdrant directly with the new embedding
            search_result = qdrant_client.search(
                collection_name=collection_name,
                query_vector=query_vector,
                limit=top_k * 3,
                query_filter={"must": [{"key": "vector_type", "match": {"value": "brand"}}]}
            )
            
            # Process results as in search_car_brand
            embedding_results = []
            unique_results = {}
            for result in search_result:
                car_brand = result.payload['car_brand']
                if car_brand not in unique_results:
                    unique_results[car_brand] = result
                    embedding_results.append(result)
        else:  # model
            # Similar for model search
            query_with_context = f"car model: {query}"
            query_vector = model_to_use.encode(query_with_context)
            
            # Call Qdrant directly
            search_result = qdrant_client.search(
                collection_name=collection_name,
                query_vector=query_vector,
                limit=top_k * 3,
                query_filter={"must": [{"key": "vector_type", "match": {"value": "model"}}]}
            )
            
            # Process results
            embedding_results = []
            unique_results = {}
            for result in search_result:
                car_model = result.payload['car_model']
                if car_model not in unique_results:
                    unique_results[car_model] = result
                    embedding_results.append(result)
        
        # Extract relevant information
        for result in embedding_results:
            if vector_type == "brand":
                text = result.payload.get("car_brand")
            else:  # model
                text = result.payload.get("car_model")
                
            # Skip if this result is already in our list from fuzzy matching
            if any(r["text"] == text for r in results):
                continue
                
            # Add to results
            results.append({
                "text": text,
                "score": result.score,
                "source": "embedding"
            })
    
    # Return top_k results, sorted by score
    return sorted(results, key=lambda x: x["score"], reverse=True)[:top_k]

# Testing car brand typo detection


In [38]:
from rapidfuzz import process as rapidfuzz_process
from rapidfuzz import fuzz
import re

# Load car data
df = pd.read_csv('car_dataset.csv')
brand_choices = list(df['car_brand'].unique())


# Filter only brand test cases
brand_eval_cases = [
    # Brands
    ("perodue", "Perodua"),
    ("perjdia", "Perodua"),
    ("[roton", "Proton"),
    ("protoon", "Proton"),
    ("toyouta", "Toyota"),
    ("hinda", "Honda"),
    ("neesun", "Nissan"),          # phonetic
    ("merc benz", "Mercedes-Benz"),
    ("benz", "Mercedes-Benz"),     # abbreviation
    ("bmww", "BMW"),
    ("bydd", "BYD"),
    ("wolkswagen", "Volkswagen"), 
    ("p495on", "Proton"),          # OCR: numbers
    ("p4tons", "Proton"),
    ("jpnda", "Honda"),            # keyboard typo
    ("protin", "Proton"),
    ("e-mas seven", "E-Mas 7"),    # number word
    
    # Extra OCR confusions
    ("t0y0ta", "Toyota"),          # 0 vs o
    ("h0nda", "Honda"),
    
    # Extra phonetic cases
    ("nisan", "Nissan"),
    ("merz", "Mercedes-Benz"),
    ("bemer", "BMW"),
    
    # Concatenation / spacing
    ("mercedesbenz", "Mercedes-Benz"),
    ("perod u a", "Perodua"),

    ("saga", "Proton"),
    ("City", "Honda"),
    ("Myvi", "Perodua"),
    ("Axia", "Perodua"),
]

print("\nTESTING BRAND TYPO DETECTION\n")

# Track stats
brand_stats = {"total": len(brand_eval_cases), "fuzzy": 0, "embedding": 0}

for query, expected in brand_eval_cases:
    # Test brand search
    results = hybrid_search(query, brand_choices, vector_type="brand", fuzzy_threshold=75, top_k=3, search_model=model)
    
    # Track which method provided the results
    if results and results[0]["source"] == "fuzzy":
        brand_stats["fuzzy"] += 1
    elif results:
        brand_stats["embedding"] += 1
        
    # Print results
    print(f"\nQuery: '{query}' (expected: {expected})")
    for i, res in enumerate(results, 1):
        match = "✓" if res["text"] == expected else " "
        print(f"  {i}. {res['text']} ({res['score']:.4f}, {res['source']}) {match}")
    
    # Visual separator
    print("-" * 60)

# Print summary stats
print("\n=== BRAND TYPO DETECTION SUMMARY ===")
print(f"Total cases: {brand_stats['total']}")
print(f"Resolved by fuzzy: {brand_stats['fuzzy']} ({brand_stats['fuzzy']/brand_stats['total']*100:.1f}%)")
print(f"Resolved by embeddings: {brand_stats['embedding']} ({brand_stats['embedding']/brand_stats['total']*100:.1f}%)")


TESTING BRAND TYPO DETECTION

Found 1 good fuzzy matches for 'perodue'

Query: 'perodue' (expected: Perodua)
  1. Perodua (0.8571, fuzzy) ✓
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings


C:\Users\aykay\AppData\Local\Temp\ipykernel_3208\3121691710.py:77: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(



Query: 'perjdia' (expected: Perodua)
  1. Perodua (0.9269, embedding) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for '[roton'

Query: '[roton' (expected: Proton)
  1. Proton (0.8333, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'protoon'

Query: 'protoon' (expected: Proton)
  1. Proton (0.9231, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'toyouta'

Query: 'toyouta' (expected: Toyota)
  1. Toyota (0.9231, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'hinda'

Query: 'hinda' (expected: Honda)
  1. Honda (0.8000, fuzzy) ✓
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings

Query: 'neesun' (expected: Nissan)
  1. Perodua (0.9028, embedding)  
  2. Nissan (0.9001, embedding) ✓
  3. Proton (0.8953, embedding)  


In [8]:
from rapidfuzz import process as rapidfuzz_process
from rapidfuzz import fuzz
import re

# Load car data
df = pd.read_csv('car_dataset.csv')
brand_choices = list(df['car_brand'].unique())


# Filter only brand test cases
brand_eval_cases = [
    # Brands
    ("merz", "Mercedes-Benz"),
]

print("\nTESTING BRAND TYPO DETECTION\n")

finetuned_model = SentenceTransformer("./model/finetuned-embedding-model-v2")
# Track stats
brand_stats = {"total": len(brand_eval_cases), "fuzzy": 0, "embedding": 0}

for query, expected in brand_eval_cases:
    # Test brand search
    results = hybrid_search(query, brand_choices, vector_type="brand", fuzzy_threshold=75, top_k=3, search_model=finetuned_model)
    
    # Track which method provided the results
    if results and results[0]["source"] == "fuzzy":
        brand_stats["fuzzy"] += 1
    elif results:
        brand_stats["embedding"] += 1
        
    # Print results
    print(f"\nQuery: '{query}' (expected: {expected})")
    for i, res in enumerate(results, 1):
        match = "✓" if res["text"] == expected else " "
        print(f"  {i}. {res['text']} ({res['score']:.4f}, {res['source']}) {match}")
    
    # Visual separator
    print("-" * 60)

# Print summary stats
print("\n=== BRAND TYPO DETECTION SUMMARY ===")
print(f"Total cases: {brand_stats['total']}")
print(f"Resolved by fuzzy: {brand_stats['fuzzy']} ({brand_stats['fuzzy']/brand_stats['total']*100:.1f}%)")
print(f"Resolved by embeddings: {brand_stats['embedding']} ({brand_stats['embedding']/brand_stats['total']*100:.1f}%)")


TESTING BRAND TYPO DETECTION

No good fuzzy matches above threshold 75, using embeddings


C:\Users\aykay\AppData\Local\Temp\ipykernel_4700\3121691710.py:77: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(



Query: 'merz' (expected: Mercedes-Benz)
  1. Mercedes-Benz (0.5894, embedding) ✓
  2. Chery (0.5816, embedding)  
  3. Perodua (0.5593, embedding)  
------------------------------------------------------------

=== BRAND TYPO DETECTION SUMMARY ===
Total cases: 1
Resolved by fuzzy: 0 (0.0%)
Resolved by embeddings: 1 (100.0%)


# Model Typo Detection

In [40]:
# Load car data
model_choices = list(df['car_model'].unique())

# Filter only model test cases
model_eval_cases = [
    # Models only
    ("bezxa", "Bezza"),
    ("bizza", "Bezza"),
    ("axla", "Axia"),
    ("aksia", "Axia"),
    ("myvee", "Myvi"),
    ("sagha", "Saga"),
    ("alzza", "Alza"),
    ("attiva", "Ativa"),
    ("vi0s", "Vios"),   # OCR: 0 vs o
    ("ciity", "City"),
    ("x5o", "X50"),    # 5 vs S vs o
    ("hilax", "Hilux"),
    ("hiluks", "Hilux"),
    ("xseventy", "X70"),           # number word
    ("cx thirty", "CX-30"),        # number word
    ("x-7o", "X70"),               # OCR: 7 vs T
    (",yvi", "Myvi"),              # OCR: , vs m
    ("sivic", "Civic"),            # phonetic

    # Extra OCR confusions
    ("v1os", "Vios"),

    # Extra phonetic cases
    ("civik", "Civic"),

    # Concatenation / spacing
    ("hondacity", "City"),

    # Word-number transcriptions
    ("x fifty", "X50"),
    ("c x thirty", "CX-30"),

    # Keyboard adjacency mistakes
    ("aqia", "Axia"),
    ("citu", "City"),
    ("hilud", "Hilux"),

    ("Proton", "Proton"),
    ("Honda", "Honda"),
    ("Myvi", "Myvi"),
    ("Perodua", "Perodua"),
    ("Chery", "Chery"),
    ("BMW", "BMW"),
]

print("\nTESTING MODEL TYPO DETECTION\n")

# Track stats
model_stats = {"total": len(model_eval_cases), "fuzzy": 0, "embedding": 0}

for query, expected in model_eval_cases:
    # Test model search
    results = hybrid_search(query, model_choices, vector_type="model", fuzzy_threshold=75, top_k=3)
    
    # Track which method provided the results
    if results and results[0]["source"] == "fuzzy":
        model_stats["fuzzy"] += 1
    elif results:
        model_stats["embedding"] += 1
        
    # Print results
    print(f"\nQuery: '{query}' (expected: {expected})")
    for i, res in enumerate(results, 1):
        match = "✓" if res["text"] == expected else " "
        print(f"  {i}. {res['text']} ({res['score']:.4f}, {res['source']}) {match}")
    
    # Visual separator
    print("-" * 60)

# Print summary stats
print("\n=== MODEL TYPO DETECTION SUMMARY ===")
print(f"Total cases: {model_stats['total']}")
print(f"Resolved by fuzzy: {model_stats['fuzzy']} ({model_stats['fuzzy']/model_stats['total']*100:.1f}%)")
print(f"Resolved by embeddings: {model_stats['embedding']} ({model_stats['embedding']/model_stats['total']*100:.1f}%)")


TESTING MODEL TYPO DETECTION

Found 1 good fuzzy matches for 'bezxa'

Query: 'bezxa' (expected: Bezza)
  1. Bezza (0.8000, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'bizza'

Query: 'bizza' (expected: Bezza)
  1. Bezza (0.8000, fuzzy) ✓
------------------------------------------------------------
Found 2 good fuzzy matches for 'axla'

Query: 'axla' (expected: Axia)
  1. Alza (0.7500, fuzzy)  
  2. Axia (0.7500, fuzzy) ✓
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings


C:\Users\aykay\AppData\Local\Temp\ipykernel_3208\3121691710.py:98: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(



Query: 'aksia' (expected: Axia)
  1. Axia (0.9441, embedding) ✓
  2. Accord (0.9311, embedding)  
  3. Attrage (0.9270, embedding)  
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings

Query: 'myvee' (expected: Myvi)
  1. Myvi (0.9518, embedding) ✓
  2. Mirage (0.9275, embedding)  
  3. Attrage (0.9226, embedding)  
------------------------------------------------------------
Found 1 good fuzzy matches for 'sagha'

Query: 'sagha' (expected: Saga)
  1. Saga (0.8889, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'alzza'

Query: 'alzza' (expected: Alza)
  1. Alza (0.8889, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'attiva'

Query: 'attiva' (expected: Ativa)
  1. Ativa (0.9091, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'vi0s'

Query: 'vi0s' (expected: 

In [41]:
# Load car data
model_choices = list(df['car_model'].unique())

# Filter only model test cases
model_eval_cases = [
    # Models only
    ("bezxa", "Bezza"),
    ("bizza", "Bezza"),
    ("axla", "Axia"),
    ("aksia", "Axia"),
    ("myvee", "Myvi"),
    ("sagha", "Saga"),
    ("alzza", "Alza"),
    ("attiva", "Ativa"),
    ("vi0s", "Vios"),   # OCR: 0 vs o
    ("ciity", "City"),
    ("x5o", "X50"),    # 5 vs S vs o
    ("hilax", "Hilux"),
    ("hiluks", "Hilux"),
    ("xseventy", "X70"),           # number word
    ("cx thirty", "CX-30"),        # number word
    ("x-7o", "X70"),               # OCR: 7 vs T
    (",yvi", "Myvi"),              # OCR: , vs m
    ("sivic", "Civic"),            # phonetic

    # Extra OCR confusions
    ("v1os", "Vios"),

    # Extra phonetic cases
    ("civik", "Civic"),

    # Concatenation / spacing
    ("hondacity", "City"),

    # Word-number transcriptions
    ("x fifty", "X50"),
    ("c x thirty", "CX-30"),

    # Keyboard adjacency mistakes
    ("aqia", "Axia"),
    ("citu", "City"),
    ("hilud", "Hilux"),

    ("Proton", "Proton"),
    ("Honda", "Honda"),
    ("Myvi", "Myvi"),
    ("Perodua", "Perodua"),
    ("Chery", "Chery"),
    ("BMW", "BMW"),
]

print("\nTESTING MODEL TYPO DETECTION\n")

# Track stats
model_stats = {"total": len(model_eval_cases), "fuzzy": 0, "embedding": 0}

for query, expected in model_eval_cases:
    # Test model search
    results = hybrid_search(query, model_choices, vector_type="model", fuzzy_threshold=75, top_k=3, search_model=finetuned_model)
    
    # Track which method provided the results
    if results and results[0]["source"] == "fuzzy":
        model_stats["fuzzy"] += 1
    elif results:
        model_stats["embedding"] += 1
        
    # Print results
    print(f"\nQuery: '{query}' (expected: {expected})")
    for i, res in enumerate(results, 1):
        match = "✓" if res["text"] == expected else " "
        print(f"  {i}. {res['text']} ({res['score']:.4f}, {res['source']}) {match}")
    
    # Visual separator
    print("-" * 60)

# Print summary stats
print("\n=== MODEL TYPO DETECTION SUMMARY ===")
print(f"Total cases: {model_stats['total']}")
print(f"Resolved by fuzzy: {model_stats['fuzzy']} ({model_stats['fuzzy']/model_stats['total']*100:.1f}%)")
print(f"Resolved by embeddings: {model_stats['embedding']} ({model_stats['embedding']/model_stats['total']*100:.1f}%)")


TESTING MODEL TYPO DETECTION

Found 1 good fuzzy matches for 'bezxa'

Query: 'bezxa' (expected: Bezza)
  1. Bezza (0.8000, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'bizza'

Query: 'bizza' (expected: Bezza)
  1. Bezza (0.8000, fuzzy) ✓
------------------------------------------------------------
Found 2 good fuzzy matches for 'axla'

Query: 'axla' (expected: Axia)
  1. Alza (0.7500, fuzzy)  
  2. Axia (0.7500, fuzzy) ✓
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings


C:\Users\aykay\AppData\Local\Temp\ipykernel_3208\3121691710.py:98: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(



Query: 'aksia' (expected: Axia)
  1. Axia (0.7067, embedding) ✓
  2. ASX (0.6800, embedding)  
  3. Alza (0.6540, embedding)  
------------------------------------------------------------
No good fuzzy matches above threshold 75, using embeddings

Query: 'myvee' (expected: Myvi)
  1. Myvi (0.6572, embedding) ✓
  2. Mirage (0.5891, embedding)  
  3. 5 Series (0.5831, embedding)  
------------------------------------------------------------
Found 1 good fuzzy matches for 'sagha'

Query: 'sagha' (expected: Saga)
  1. Saga (0.8889, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'alzza'

Query: 'alzza' (expected: Alza)
  1. Alza (0.8889, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'attiva'

Query: 'attiva' (expected: Ativa)
  1. Ativa (0.9091, fuzzy) ✓
------------------------------------------------------------
Found 1 good fuzzy matches for 'vi0s'

Query: 'vi0s' (expected: Vios)